basic imports

In [6]:
import numpy as np
import pandas as pd


 initialization of dataframe

In [10]:
DATA_PATH = "../data/raw/electricity.csv"
df = pd.read_csv(DATA_PATH)


a basic test of the dataframe

In [11]:
df.head()

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0,b'2',0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,b'UP'
1,0.0,b'2',0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,b'UP'
2,0.0,b'2',0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,b'UP'
3,0.0,b'2',0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,b'UP'
4,0.0,b'2',0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,b'DOWN'


Some data plotting with matplotlib

In [ ]:
import matplotlib.pyplot as plt


days = df['day'].unique()

plt.figure(figsize=(10,6))

for day in days:
    subset = df[df['day'] == day]
    plt.hist(subset['nswdemand'], bins=50, alpha=0.4, label=day)

plt.xlabel("NSW Electricity Demand")
plt.ylabel("Frequency")
plt.title("Distribution of NSW Electricity Demand by Day")
plt.legend()
# plt.show()

In [ ]:
plt.figure(figsize=(10,6))
df.boxplot(column='nswdemand', by='day', grid=False)
plt.xlabel("Day of Week")
plt.ylabel("NSW Electricity Demand")
plt.title("Electricity Demand by Day of Week")
plt.suptitle("") 
plt.show()

preparing dataframe for model

In [18]:
X_model_1 = df.drop(columns=['nswdemand'])
y_model_1 = df['nswdemand']


Encode data in a more functional manner with the first model

In [19]:
# Convert bytes to strings first
X_model_1['day'] = X_model_1['day'].str.decode('utf-8')
X_model_1['class'] = X_model_1['class'].str.decode('utf-8')

# One-hot encode
X_model_1 = pd.get_dummies(X_model_1, columns=['day', 'class'])

train model

In [20]:
from sklearn.model_selection import train_test_split

X_model_1_train, X_model_1_test, y_model_1_train, y_model_1_test = train_test_split(
    X_model_1, y_model_1, test_size=0.2, random_state=42
)

In [21]:
print(X_model_1_train.shape, X_model_1_test.shape)
print(y_model_1_train.shape, y_model_1_test.shape)

(36249, 6) (9063, 6)
(36249,) (9063,)


In [25]:
from sklearn.ensemble import RandomForestRegressor

# Create the model
model_1 = RandomForestRegressor(n_estimators=100, random_state=42)

In [26]:
model_1.fit(X_model_1_train, y_model_1_train)


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


make some predictions

In [27]:
y_model_1_pred = model_1.predict(X_model_1_test)


In [28]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

mse_1 = mean_squared_error(y_model_1_test, y_model_1_pred)
rmse_1 = np.sqrt(mse_1)
r2_1 = r2_score(y_model_1_test, y_model_1_pred)

print("RMSE:", rmse_1)
print("R²:", r2_1)

RMSE: 0.03399612640495005
R²: 0.9568581253712433
